In [ ]:
# Import DeepInsight
import sys
sys.path.insert(0, "/home/marx/Documents/Github/DeepInsight")
import deepinsight
# Choose GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import h5py
%load_ext autoreload
%autoreload 2

In [ ]:
fp_deepinsight = './test.h5'
input_length = int(3e5)
input_channels = 5
sampling_rate = 30000
input_output_ratio = 100

np.random.seed(0)
rand_input = np.sin(np.random.rand(int(input_length), input_channels))
rand_input_timesteps = np.arange(0, input_length)
rand_output = np.random.rand(input_length // input_output_ratio)
rand_timesteps = np.arange(0, input_length, input_output_ratio)

print(rand_input[0,0])
print(rand_input_timesteps[0:10])
print(rand_output[0])
print(rand_timesteps[0:10])

In [ ]:
# Transform raw data to frequency domain
deepinsight.preprocess.preprocess_input(fp_deepinsight, rand_input, sampling_rate=sampling_rate)


In [ ]:
hdf5_file = h5py.File(fp_deepinsight, mode='r')

# Get size of wavelets
input_wavelets = hdf5_file['inputs/wavelets']
# Check statistics of wavelets
np.testing.assert_almost_equal(np.mean(input_wavelets), 0.048329726)
np.testing.assert_almost_equal(np.std(input_wavelets), 0.032383125)
np.testing.assert_almost_equal(np.median(input_wavelets), 0.04608967)
np.testing.assert_almost_equal(np.max(input_wavelets), 0.40853173)
np.testing.assert_almost_equal(np.min(input_wavelets), 1.6544704e-05)

In [ ]:
np.min(input_wavelets)

In [ ]:
# Prepare outputs
deepinsight.preprocess.preprocess_output(fp_deepinsight, rand_input_timesteps, rand_output,
                                           rand_timesteps)

In [ ]:
# Define loss functions and train model
loss_functions = {'output_aligned' : 'mse'}
loss_weights = {'output_aligned' : 1}
user_opts = {'epochs' : 2, 'steps_per_epoch' : 10, 'validation_steps' : 10, 'log_output' : False, 'save_model' : False}

deepinsight.train.run_from_path(fp_deepinsight, loss_functions, loss_weights, user_opts)

In [ ]:
# Get loss and shuffled loss for influence plot, both is also stored back to HDF5 file
losses, output_predictions, indices = deepinsight.analyse.get_model_loss(fp_deepinsight, stepsize=10)

In [ ]:
shuffled_losses = deepinsight.analyse.get_shuffled_model_loss(fp_deepinsight, axis=1,stepsize=10)

In [ ]:
deepinsight.visualize.plot_residuals(fp_deepinsight, frequency_spacing=2,
                                     output_names=['output_aligned'])

In [ ]:
# from tqdm.notebook import tqdm
# import time
# for i1 in tqdm(range(5), ncols='50%'):
#     for i2 in tqdm(range(30), leave=False, ncols='25%'):
#         # do something, e.g. sleep
#         time.sleep(0.1)